In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#hannes oli siin, kas teised näevad seda?

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Network Science/project/bot_detection_data.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Network Science/project/bot_detection_data.csv'

In [ ]:
df.head()

In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
df.describe()

# Data description
There is total of 11 columns and 50 000 rows of data. Each row represents 1 tweet along with its info. The data takes up about 3.9+ MB of memory.
### Columns and description:
Info was gotten from the Readme.md file from the dataset origin
- User ID: Unique identifier for each user in the dataset.
- Username: The username associated with the user.
- Tweet: The text content of the tweet.
- Retweet Count: The number of times the tweet has been retweeted.
- Mention Count: The number of mentions in the tweet.
- Follower Count: The number of followers the user has.
- Verified: A boolean value indicating whether the user is verified or not.
- Bot Label: A label indicating whether the user is a bot (1) or not (0).
- Location: The location associated with the user.
- Created At: The date and time when the tweet was created.
- Hashtags: The hashtags associated with the tweet.

There are some Nan values in the Hastags columns, but these are not errors in our case, these describe a tweet, where the user did not select/write any hashtags. Thus these Nan values are not removed.

- From the data describe function, we can see that the median tweet got 50 retweets. The std is 29 for retweet count (pärast täpsuustada!!). The minimum number of retweet count is 0 and the maximum is 100.
- The median tweet got 2.5 mentions. The std is 1.7 for mention count. The minimum number of mention count is 0 and the maximum is 5.
- The median tweet user has got 4988.6 followers. The std is 2878.7 for follower count. The minimum number of follower count is 0 and the maximum is 10000.

In [ ]:
df['Bot Label'].value_counts()

There is total of 25018 tweets by bots and 24982 tweets by not bots, aka real human tweets. It can be summarised as 50% tweets by bots and 50% tweets by humans.

In [ ]:
df['User ID'].unique().size

In [ ]:
df['Username'].unique().size

As the number of unique user IDs is the same as the total number of tweets in our dataset, this means that we have only 1 tweet from each user. This leaves out a possability of using the number of tweets per user. As the number of unique usernames is smaller than the total number of tweets, it might seem as we have many tweets per user, but this turned out to be not true based on the info gotten from the number of unique user IDs.

In [ ]:
df['Verified'].value_counts()

We have a total of 25004 tweets from verified accounts and 24996 tweets from not verified accounts. To summarize, half of the tweets came from a verified account, the other half came from a not verified account.

Lets post a hypothesis, most of the tweets that came from a not verified account, are from bots.

In [ ]:
df[(df['Verified'] == False) & (df['Bot Label'] == 1)].shape

Turns out, 12540 tweets did fit our hypothesis, this is half of the 25018 bots we wanted to catch, this alone will not be enough, but might help us later.

Lets have a look at where are the tweets coming from.

In [ ]:
df['Location'].value_counts()

In [ ]:
location_counts = df['Location'].value_counts()
location_counts.shape

Lets see how many of the most popular location tweets are from bots?

In [ ]:
df[(df['Location'] == 'South Michael') & (df['Bot Label'] == 1)].shape

Less than half this time. Lets look how many of the tweets came from places where the location count is 1.

In [ ]:
filtered_df = df[df['Location'].isin(location_counts[location_counts == 1].index)]
filtered_df.shape

In [ ]:
filtered_df[(filtered_df['Bot Label'] == 1)].shape

There is a total of 16295 tweets that have a location with the count of 1. Around half of these are from bots. It looks like the bots tweets are evenly distributed with locations.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create bins for retweet counts
bin_edges = np.arange(0, 101, 10)

# Create a histogram for bot tweets (Bot Label = 1)
bot_tweets = df[df['Bot Label'] == 1]['Retweet Count']
plt.hist(bot_tweets, bins=bin_edges, alpha=0.5, label='Bot', color='red')

# Create a histogram for non-bot tweets (Bot Label = 0)
non_bot_tweets = df[df['Bot Label'] == 0]['Retweet Count']
plt.hist(non_bot_tweets, bins=bin_edges, alpha=0.5, label='Non-Bot', color='blue')

# Customize the plot
plt.xlabel('Retweet Count')
plt.ylabel('Frequency')
plt.title('Distribution of Retweet Counts for Bot and Non-Bot Tweets (0-100)')
plt.xticks(bin_edges)  # Set x-axis ticks to match bins
plt.legend()

# Display the graph
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create bins for retweet counts 0-5
bins = range(0, 6)  # Bins: 0, 1, 2, 3, 4, 5

# Create a histogram for bot tweets (Bot Label = 1)
bot_tweets = df[df['Bot Label'] == 1]['Mention Count']
plt.hist(bot_tweets, bins=bins, alpha=0.5, label='Bot', color='red')

# Create a histogram for non-bot tweets (Bot Label = 0)
non_bot_tweets = df[df['Bot Label'] == 0]['Mention Count']
plt.hist(non_bot_tweets, bins=bins, alpha=0.5, label='Non-Bot', color='blue')

# Customize the plot
plt.xlabel('Mention Count')
plt.ylabel('Frequency')
plt.title('Distribution of Mention Counts for Bot and Non-Bot Tweets (0-5)')
plt.xticks(bins)  # Set x-axis ticks to match bins
plt.legend()

# Display the graph
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create bins for retweet counts
bin_edges = np.arange(0, 1001, 50)

# Create a histogram for bot tweets (Bot Label = 1)
bot_tweets = df[df['Bot Label'] == 1]['Follower Count']
plt.hist(bot_tweets, bins=bin_edges, alpha=0.5, label='Bot', color='red')

# Create a histogram for non-bot tweets (Bot Label = 0)
non_bot_tweets = df[df['Bot Label'] == 0]['Follower Count']
plt.hist(non_bot_tweets, bins=bin_edges, alpha=0.5, label='Non-Bot', color='blue')

# Customize the plot
plt.xlabel('Follower Count')
plt.ylabel('Frequency')
plt.title('Distribution of Follower Counts for Bot and Non-Bot Tweets (0-100)')
plt.xticks(bin_edges)  # Set x-axis ticks to match bins
plt.legend()

# Display the graph
plt.show()

In [ ]:
import networkx as nx
import pandas as pd

G = nx.DiGraph()

for idx, row in df.iterrows():
    user_id = row['User ID']
    user_label = row['Bot Label']
    user_followers = row['Follower Count']

    # Add the node if not already present
    if not G.has_node(user_id):
        G.add_node(user_id,
                   label=row['Username'],
                   is_bot=user_label,
                   followers=user_followers)

    # If mentions exist, create edges
    mentions = row['Tweet'].split('@')[1:]  # crude way to extract mentions
    for mention in mentions:
        mentioned_user = mention.split()[0].strip(',:.!?')  # clean up username
        G.add_edge(user_id, mentioned_user)


In [ ]:
import matplotlib.pyplot as plt

colors = ['red' if G.nodes[n]['is_bot'] == 1 else 'blue' for n in G.nodes]
sizes = [G.nodes[n]['followers'] / 10 + 10 for n in G.nodes]  # scale size

plt.figure(figsize=(12, 12))
pos = nx.spring_layout(G, k=0.1)  # Force-directed layout

nx.draw(G, pos, node_color=colors, node_size=sizes, with_labels=False, edge_color='gray', alpha=0.6)
plt.title('User Interaction Network: Bots vs Humans')
plt.show()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

# Load data
#df = pd.read_csv('your_data.csv')
df2 = df.copy()
# Basic cleaning
df2['verified'] = df['Verified'].astype(int)
df2['tweet_length'] = df['Tweet'].astype(str).apply(len)
df2['hashtag_count'] = df['Hashtags'].astype(str).apply(lambda x: len(x.split(',')) if x != 'nan' else 0)

# Extract time features
df2['created_at'] = pd.to_datetime(df['Created At'])
df2['tweet_hour'] = df2['created_at'].dt.hour
df2['tweet_weekday'] = df2['created_at'].dt.weekday

# TF-IDF
vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = vectorizer.fit_transform(df2['Tweet'].astype(str))

# Combine features
X = pd.concat([
    df2[['Follower Count', 'verified', 'Retweet Count', 'Mention Count', 'tweet_length', 'hashtag_count', 'tweet_hour', 'tweet_weekday']].reset_index(drop=True),
    pd.DataFrame(tfidf_matrix.toarray())
], axis=1)

X.columns = X.columns.astype(str)  #

y = df2['Bot Label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

importances = clf.feature_importances_
indices = np.argsort(importances)[-8:]  # top 10

plt.figure(figsize=(10, 5))
plt.title("Top 10 Feature Importances")
plt.barh(range(len(indices)), importances[indices], align="center")
plt.yticks(range(len(indices)), [X.columns[i] for i in indices])
plt.xlabel("Importance")
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000))
])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

print(classification_report(y_test, y_pred))